In [2]:
import pandas as pd
import numpy as np
import time
import re
import nltk
from time import sleep
from random import randint
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from nltk import word_tokenize 
from nltk.util import ngrams
stopwords = nltk.corpus.stopwords.words('english')
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [3]:
print(pd.__version__)
print(np.__version__)
print(re.__version__)
print(nltk.__version__)

1.2.0
1.19.5
2.2.1
3.4.5


In [4]:
#Output File
scraped_data = 'Scraped_Data.csv'

In [5]:
''''Keywords for various brands

   For ZSEV:
  if 'power' in i or 'worth' in i or 'tech' in i or 'electric' in i or 'performance' in i or 'battery' in i or 'charg' in i or 'maintenance' in i or 'eco' in i  or 'styl' in i or 'safe' in i or 'comf' in i or 'space' in i or 'bumper' in i or 'zsev' in i or 'electric' in i or 'panel' in i or 'ventilated' in i or 'organic' in i or 'green' in i or 'help' in i or 'assistance' in i or 'env' in i or 'expensive' in i or 'poor' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i or 'EV' in i or 'issue' in i or 'sales' in i or 'service' in i or 'engine' in i or 'clutch' in i or 'surp' in i:
   
   For TATA Harrier
  if 'comf' in i or 'space' in i or 'ventilated' in i or 'expensive' in i or 'tata' in i or 'harrier' in i or 'poor' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i:
  
   For  Hyundai Creta:
  if 'comf' in i or 'space' in i or 'ventilated' in i or 'expensive' in i or 'poor' in i or 'creta' in i or 'hyundai' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i:

   For Hector & Hector Plus:
  if 'comf' in i or 'sharp' in i or 'smooth' in i or 'amazing' in i or 'cheap' in i or 'bad' in i or 'space' in i or 'ventilated' in i or 'human thing' in i or 'expensive' in i or 'poor' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i or 'experience' in i or if 'ada' in i or 'nice' in i or 'safety' in i or 'happy' in i or 'engine' in i or 'good' in i or 'look' in i or 'front' in i or 'suv' in i:
 
   For Gloster:
  if 'comf' in i or 'gloster' in i or 'built' in i or 'engine' in i or 'gear' in i or 'big' in i or 'perf' in i or 'reliabl' in i or 'product' in i or 'size' in i or 'feature' in i or 'space' in i or 'ventilated' in i in i or 'expensive' in i or 'poor' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i:
  '''

"'Keywords for various brands\n\n   For ZSEV:\n  if 'power' in i or 'worth' in i or 'tech' in i or 'electric' in i or 'performance' in i or 'battery' in i or 'charg' in i or 'maintenance' in i or 'eco' in i  or 'styl' in i or 'safe' in i or 'comf' in i or 'space' in i or 'bumper' in i or 'zsev' in i or 'electric' in i or 'panel' in i or 'ventilated' in i or 'organic' in i or 'green' in i or 'help' in i or 'assistance' in i or 'env' in i or 'expensive' in i or 'poor' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i or 'EV' in i or 'issue' in i or 'sales' in i or 'service' in i or 'engine' in i or 'clutch' in i or 'surp' in i:\n   \n   For TATA Harrier\n  if 'comf' in i or 'space' in i or 'ventilated' in i or 'expensive' in i or 'tata' in i or 'harrier' in i or 'poor' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i:\n  \n   For

In [6]:
'''Scraping Multiple Webpages
Websites :

Cardekho  - 'https://www.cardekho.com/mg/hector/user-reviews'  + '/' + str(page) + '?subtab=latest'
Cartrade  - 'https://www.cartrade.com/mg-cars/hector/reviews/hector-expert-reviews/207079.html'
ZigWheels - 'https://www.zigwheels.com/reviews-advice/reviews/mg-zs-ev-first-drive-review/36570/#leadform'
Team-BHP  - 'https://www.team-bhp.com/forum/official-new-car-reviews/214645-mg-gloster-official-review-'+ str(page) +'.html'
Carwale   - 'https://www.carwale.com/mg-cars/hector/news/'+ 'page/' + str(page) +'/'
AutoCar   - 'https://www.autocarindia.com/car-news/0'+str(page)
TOI       - 'https://auto.timesofindia.com/news'+ '/' + str(page)
Facebook  - 'https://www.facebook.com/MGMotorIN/reviews/?ref=page_internal'
'''
pages = np.arange(1, 4, 1)
data=[]

for page in pages:
    opt = webdriver.ChromeOptions()
    opt.add_argument("--disable-notifications")
    opt.add_argument("--disable-popup-blocking")
    opt.add_argument("start-maximized")
    opt.add_argument('disable-infobars')
    page = 'https://www.cardekho.com/mg/hector/user-reviews'  + '/' + str(page) + '?subtab=latest'
    path = r'C:\\Users\\tanisha16\\Downloads\\Chromedriver'
    driver = webdriver.Chrome(executable_path = path,options=opt)
    driver.get(page)
Brand=[]
for i in driver.find_elements_by_tag_name("div"): Brand.append(i.text)

In [7]:
Brand_Str = ' '.join([str(elem) for elem in Brand]) 

In [8]:
'''list of unwanted words'''
newStopWords = ['view','likeswrite','like','comment','Share','Jan','Feb','Mar','Apr','Jun','Jul','May','user','review','lakh','read','story','By','Views','Read','More','Likes','Comments','LikesWrite','On','Comment','I','News','Reviews']

In [9]:
wordnet = WordNetLemmatizer()
sentences = nltk.sent_tokenize(Brand_Str)
corpus=[]
for i in range(len(sentences)):
    review = re.sub('[^a-zA-Z]',' ', sentences[i])
    #review = review.lower()
    review = review.split()
    stopwords.extend(newStopWords)
    review = [wordnet.lemmatize(word) for word in review if word not in stopwords]   #list comprehension
    review = ' '.join(review)
    corpus.append(review)

In [10]:
final_corpus=[]
for i in corpus:
    if 'comf' in i or 'space' in i or 'ventilated' in i or 'human thing' in i or 'expensive' in i or 'poor' in i or 'luxurious' in i or 'mileage' in i or 'maintenance fee' in i or 'not' in i or 'look' in i or 'dealer' in i or 'best' in i:
        final_corpus.append(i)
final_corpus
reviews = [] 
[reviews.append(x) for x in final_corpus if x not in reviews]
reviews
from pandas import DataFrame
df = DataFrame (reviews,columns=['Review'])
df
df.drop_duplicates(subset ="Review", 
                     keep = False, inplace = True)

In [11]:
final=' '.join([str(elem) for elem in final_corpus]) 
words = nltk.word_tokenize(final)

words

pos_tags = nltk.pos_tag(words)
chunks = nltk.ne_chunk(pos_tags, binary=False)
entities=[]
labels=[]
for chunk in chunks:
    if hasattr(chunk,'label'):
        entities.append(' '.join(c[0] for c in chunk))
        labels.append(chunk.label())
    else:
        entities.append(' ')
        labels.append(' ')
#entities_labels=list(set(zip(entities, labels)))     #can use this instead of dict too
#dfe = pd.DataFrame(entities_labels)
#dfe.columns = ['Entities','Labels']

In [12]:
df['Refined_Review'] = df['Review'].str.replace('|'.join(map(re.escape, entities)), ' ') 

C:\Users\Tanisha16\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [13]:
'''Assigning Polarity - By Vader'''
analyzer = SentimentIntensityAnalyzer()
df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['Refined_Review']]
df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['Refined_Review']]
df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['Refined_Review']]
df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['Refined_Review']]

score=[]
for value in df['compound']:
    if value<0:
        score.append(-1)
    elif value>0:
        score.append(1)
    else:
        score.append(0)
df['Polarity_Vader'] = score       

df.drop(['compound','pos','neg','neu'],axis=1,inplace=True)

df['Refined_Review']=df['Refined_Review'].apply(lambda x: x.lower())
df['Refined_Review'] = df['Refined_Review'].apply(lambda x: x.strip())

In [14]:
#df.drop_duplicates(subset ="Refined_Review",  keep = False, inplace = True)
df['Source'] = 'Cardekho'
df['Brand']  = 'Hector'
df['Campaign'] = 'Paytm Voucher'
df = df.reset_index()
df = df[['Source','Brand','Campaign','Review','Refined_Review','Polarity_Vader']]

In [15]:
df

,Source,Brand,Campaign,Review,Refined_Review,Polarity_Vader
0,Cardekho,Hector,Paytm Voucher,The best car Extremely luxurious comfortable d...,the best car extremely luxurious comfortable d...,1
1,Cardekho,Hector,Paytm Voucher,Anubhav Singh Great Experience If looking buy ...,if looking buy better feel just amazing ...,1
2,Cardekho,Hector,Paytm Voucher,You love comfort car enjoy city driving highway,you love comfort car enjoy city driving highway,1
3,Cardekho,Hector,Paytm Voucher,decided buy MG HECTOR This car absolutely perf...,decided buy hector this car absolutely perfe...,1
4,Cardekho,Hector,Paytm Voucher,The car much comfortable drive good legroom sp...,the car much comfortable drive good legroom sp...,1
5,Cardekho,Hector,Paytm Voucher,The ride quality excellent cannot asking,the ride quality excellent cannot asking,1
6,Cardekho,Hector,Paytm Voucher,Altho Rahul D Hector Good Than Hector Plus It ...,plus it best price lakh,1
7,Cardekho,Hector,Paytm Voucher,MG Hector plus congested Hector best,plus congested best,1
8,Cardekho,Hector,Paytm Voucher,Raja Upadhyay Nothing Like MG Hector Best car ...,like best car styling comfort convenience ...,1
9,Cardekho,Hector,Paytm Voucher,Krusna Prasad Jena Mileage Of CVT Hector Very ...,poor mileage kmpl city around kmpl highway pet...,-1


## Twitter Sentiment  Analysis

In [16]:
from textblob import TextBlob
import sys
import tweepy
import pandas as pd
import numpy as np
import os
import nltk
import re
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

consumerKey = "AN7fgNvfsoHqscSFi5fyk57c9"
consumerSecret = "B10n6mF85fPMuPRAshL2Wu5vDlC9ZknpWwLdyWw7SdvuIZJKDf"
accessToken = "1356135695660670979-XeyEUcYO6oZgYV6lc1E9j7AwWskJxT"
accessTokenSecret = "xJsDAmlwVcRbqfwfi3QnmKNa9sHkPJR0unXIeAyzmbI24"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)                                     #using tweepy to get text after the completion of authorization

In [17]:
print(pd.__version__)
print(np.__version__)
print(re.__version__)
print(nltk.__version__)
print(tweepy.__version__)

1.2.0
1.19.5
2.2.1
3.4.5
3.10.0


In [18]:
''''Hashtags/Campaigns for various brands

   For ZSEV: ChangeWhatYouCan, mg zsev
   
   For TATA Harrier: tataharrier, 1WithMyHarrier, ClubHarrier, Harrians
  
   For  Hyundai Creta: hyundaicreta, hyundai creta

   For Hector: ItsAHumanThing, StoriesOfMG, mg hector
   
   For Hector Plus: Hector Plus, hector plus
 
   For Gloster: TheSelflessCar, mg gloster
  '''

"'Hashtags/Campaigns for various brands\n\n   For ZSEV: ChangeWhatYouCan, mg zsev\n   \n   For TATA Harrier: tataharrier, 1WithMyHarrier, ClubHarrier, Harrians\n  \n   For  Hyundai Creta: hyundaicreta, hyundai creta\n\n   For Hector: ItsAHumanThing, StoriesOfMG, mg hector\n   \n   For Hector Plus: Hector Plus, hector plus\n \n   For Gloster: TheSelflessCar, mg gloster\n  "

In [19]:
#Sentiment Analysis
#Segregating the tweets into positive negative and neutral
def percentage(part,whole):
    return 100 * float(part)/float(whole)
keyword = input("Please enter keyword or hashtag to search: ")
noOfTweet = int(input ("Please enter how many tweets to analyze: "))
tweets = tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

Please enter keyword or hashtag to search: mghector
Please enter how many tweets to analyze: 20


In [20]:
for tweet in tweets:
 
 #print(tweet.text)
    tweet_list.append(tweet.text)
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
 
    if neg > pos:
        negative_list.append(tweet.text)
        negative += 1
    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
 
    elif pos == neg:                                  
        neutral_list.append(tweet.text)
        neutral += 1
positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [21]:
#NER
fin=' '.join([str(elem) for elem in tweet_list]) 
words = nltk.word_tokenize(fin)

words

pos_tags = nltk.pos_tag(words)
chunks = nltk.ne_chunk(pos_tags, binary=False)
entities=[]
labels=[]
for chunk in chunks:
    if hasattr(chunk,'label'):
        entities.append(' '.join(c[0] for c in chunk))
        labels.append(chunk.label())
    else:
        entities.append(' ')
        labels.append(' ')

In [22]:
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print("total number: ",len(tweet_list))
print("positive number: ",len(positive_list))
print("negative number: ", len(negative_list))
print("neutral number: ",len(neutral_list))

total number:  15
positive number:  5
negative number:  1
neutral number:  9


In [23]:
tweet_list.drop_duplicates(inplace = True)
tweet = tweet_list
tw_list = pd.DataFrame(tweet)
tw_list["Review"] = tw_list[0]
tw_list['Refined_Review'] = tw_list['Review'].str.replace('|'.join(map(re.escape, entities)), ' ') 
tw_list['Refined_Review'] = tw_list['Refined_Review'].str.replace('[^\w\s]','')
tw_list['Refined_Reviewt'] = tw_list['Refined_Review'].str.lower()

df1=tw_list

analyzer = SentimentIntensityAnalyzer()
df1['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df1['Refined_Review']]
df1['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df1['Refined_Review']]
df1['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df1['Refined_Review']]
df1['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df1['Refined_Review']]

score=[]
for value in df1['compound']:
    if value<0:
        score.append(-1)
    elif value>0:
        score.append(1)
    else:
        score.append(0)
df1['Polarity_Vader'] = score       

df1.drop(['compound','pos','neg','neu',0],axis=1,inplace=True)

C:\Users\Tanisha16\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
C:\Users\Tanisha16\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [24]:
#df.drop_duplicates(subset ="Refined_Review",  keep = False, inplace = True)
df1['Source'] = 'Twitter'
df1['Brand']  = 'Hector'
df1['Campaign'] = 'ItsAHumanThing'
df1 = df1.reset_index()
df1 = df1[['Source','Brand','Campaign','Review','Refined_Review','Polarity_Vader']]

In [25]:
df1

,Source,Brand,Campaign,Review,Refined_Review,Polarity_Vader
0,Twitter,Hector,ItsAHumanThing,"Say #MgHector ,#TataHarrier, #JeepCompass and ...",Say Mg and are brothers as they share t...,1
1,Twitter,Hector,ItsAHumanThing,Base model price of #Alcazar should be 1 lakh ...,Base model price of should be 1 lakh more th...,0
2,Twitter,Hector,ItsAHumanThing,@MGMotorIn has increased prices of the entire ...,has increased prices of the entire and r...,1
3,Twitter,Hector,ItsAHumanThing,"@MGMotorIn has now sold over 50,000 units of t...",has now sold over 50000 units of the SUVs ...,0
4,Twitter,Hector,ItsAHumanThing,#seltos #kia #kiaseltos #creta #k #kiamotors #...,seltos kia kiaseltos creta k kiamotors xuv for...,-1
5,Twitter,Hector,ItsAHumanThing,#MGHector #Cars \nMG Motor India has raised He...,Cars \n Motor has raised and Plus SU...,0
6,Twitter,Hector,ItsAHumanThing,We concluded the weekend with a blast when thi...,We concluded the weekend with a blast when thi...,-1
7,Twitter,Hector,ItsAHumanThing,@hormazdsorabjee When cars like Toyota didn’t ...,hormazdsorabjee When cars like didnt upgrade...,-1
8,Twitter,Hector,ItsAHumanThing,"This Easter, let there be hope, peace and love...",This Easter let there be hope peace and love T...,1
9,Twitter,Hector,ItsAHumanThing,माझ्या #MGHector ला नाव ठेवायच नाही👊\n\n#लाडका...,मझय ल नव ठवयच नह\n\nलडक आण त य दनच गषट मल म...,0


In [37]:
df = df.append(df1)

In [38]:
df.to_csv(scraped_data, index=False)